In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
import random
import math

In [ ]:
def generate_landmark_data(num_samples_per_gesture=100, noise_level=0.03):
    gestures = ['fist', 'open_palm', 'pointing_index', 'thumb_up']
    num_landmarks = 21
    data = []
    labels = []

    # Simplified base coordinates (relative to wrist at 0,0)
    base_coords = {
        'fist': np.random.rand(num_landmarks, 2) * 0.2, # Tightly clustered
        'open_palm': np.array([
            [0, 0], # Wrist
            [-0.1, 0.1], [-0.2, 0.2], [-0.3, 0.3], [-0.4, 0.4], # Thumb
            [0, 0.2], [0, 0.4], [0, 0.6], [0, 0.8], # Index
            [0.1, 0.2], [0.15, 0.4], [0.2, 0.6], [0.25, 0.8], # Middle
            [0.2, 0.18], [0.3, 0.38], [0.4, 0.58], [0.5, 0.78], # Ring
            [0.3, 0.15], [0.45, 0.35], [0.6, 0.55], [0.7, 0.75] # Pinky
        ]),
        'pointing_index': np.array([
            [0, 0], # Wrist
            [-0.1, 0.1], [-0.15, 0.15], [-0.2, 0.2], [-0.25, 0.25], # Thumb (closed)
            [0, 0.2], [0, 0.4], [0, 0.6], [0, 0.8], # Index (extended)
            [0.1, 0.15], [0.15, 0.2], [0.2, 0.25], [0.25, 0.3], # Middle (closed)
            [0.15, 0.13], [0.25, 0.18], [0.35, 0.23], [0.45, 0.28], # Ring (closed)
            [0.2, 0.1], [0.35, 0.15], [0.5, 0.2], [0.6, 0.25] # Pinky (closed)
        ]),
        'thumb_up': np.array([
            [0, 0], # Wrist
            [-0.1, 0.1], [-0.15, 0.2], [-0.2, 0.3], [-0.25, 0.4], # Thumb (extended up)
            [0.05, 0.1], [0.08, 0.15], [0.1, 0.2], [0.12, 0.25], # Index (closed)
            [0.1, 0.1], [0.15, 0.15], [0.2, 0.2], [0.25, 0.25], # Middle (closed)
            [0.15, 0.08], [0.25, 0.13], [0.35, 0.18], [0.45, 0.23], # Ring (closed)
            [0.2, 0.05], [0.35, 0.1], [0.5, 0.15], [0.6, 0.2] # Pinky (closed)
        ])
    }
    
    # Ensure fist base coords have correct shape
    base_coords['fist'] = np.vstack([ [0,0], base_coords['fist'][1:]]) # Keep wrist at 0,0
    if base_coords['fist'].shape[0] < num_landmarks:
         base_coords['fist'] = np.vstack([base_coords['fist'], np.random.rand(num_landmarks - base_coords['fist'].shape[0] , 2) * 0.15])
    elif base_coords['fist'].shape[0] > num_landmarks:
         base_coords['fist'] = base_coords['fist'][:num_landmarks,:]

    for gesture in gestures:
        base = base_coords[gesture]
        for _ in range(num_samples_per_gesture):
            # Add noise
            noisy_coords = base + np.random.normal(0, noise_level, base.shape)
            # Normalize relative to wrist (landmark 0)
            wrist = noisy_coords[0, :].copy()
            normalized_coords = noisy_coords - wrist
            # Optional: Scale normalization (e.g., by distance between wrist and middle finger MCP)
            scale_p1 = normalized_coords[0,:] # Wrist
            scale_p2 = normalized_coords[9,:] # Middle finger MCP
            dist = np.linalg.norm(scale_p1 - scale_p2)
            if dist > 1e-6: # Avoid division by zero
                 normalized_coords /= dist
            
            # Flatten (x0, y0, x1, y1, ...)
            flattened_coords = normalized_coords.flatten()
            data.append(flattened_coords)
            labels.append(gesture)

    # Create DataFrame
    columns = []
    for i in range(num_landmarks):
        columns.extend([f'landmark_{i}_x', f'landmark_{i}_y'])
    
    df = pd.DataFrame(data, columns=columns)
    df['gesture_label'] = labels
    return df

# Generate the data
df_landmarks = generate_landmark_data(num_samples_per_gesture=200, noise_level=0.02)


In [ ]:
print("Data Head:")
print(df_landmarks.head())
print("\nData Info:")
df_landmarks.info()
print("\nData Description:")
print(df_landmarks.describe())
print("\nGesture Counts:")
print(df_landmarks['gesture_label'].value_counts())

In [ ]:
sns.set(style="whitegrid")

# Plot distribution of a few key landmark coordinates
key_landmarks_indices = [4, 8, 12, 16, 20] # Fingertips
coord_cols = []
for idx in key_landmarks_indices:
    coord_cols.extend([f'landmark_{idx}_x', f'landmark_{idx}_y'])

plt.figure(figsize=(15, 10))
for i, col in enumerate(coord_cols):
    plt.subplot(len(key_landmarks_indices), 2, i + 1)
    sns.histplot(data=df_landmarks, x=col, hue='gesture_label', kde=True, bins=30)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

# Box plots for the same landmarks
plt.figure(figsize=(15, 10))
for i, col in enumerate(coord_cols):
    plt.subplot(len(key_landmarks_indices), 2, i + 1)
    sns.boxplot(data=df_landmarks, x='gesture_label', y=col)
    plt.title(f'Box Plot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
def plot_gesture_landmarks(df, gesture_label, num_samples=5):
    gesture_data = df[df['gesture_label'] == gesture_label].sample(min(num_samples, len(df[df['gesture_label'] == gesture_label])))
    num_landmarks = 21
    
    plt.figure(figsize=(6, 6))
    plt.title(f'Sample Landmarks for Gesture: {gesture_label}')
    plt.xlabel('X coordinate (normalized)')
    plt.ylabel('Y coordinate (normalized)')
    plt.gca().set_aspect('equal', adjustable='box')
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(gesture_data)))
    
    for idx, (_, sample) in enumerate(gesture_data.iterrows()):
        landmarks_x = [sample[f'landmark_{i}_x'] for i in range(num_landmarks)]
        landmarks_y = [sample[f'landmark_{i}_y'] for i in range(num_landmarks)]
        plt.scatter(landmarks_x, landmarks_y, alpha=0.6, label=f'Sample {idx+1}' if idx < 5 else "", color=colors[idx])
        
        # Optional: Draw connections (like MediaPipe)
        connections = [
            (0, 1), (1, 2), (2, 3), (3, 4), # Thumb
            (0, 5), (5, 6), (6, 7), (7, 8), # Index
            (0, 9), (9, 10), (10, 11), (11, 12), # Middle
            (0, 13), (13, 14), (14, 15), (15, 16), # Ring
            (0, 17), (17, 18), (18, 19), (19, 20), # Pinky
            (5, 9), (9, 13), (13, 17) # Palm
        ]
        for conn in connections:
            p1_idx, p2_idx = conn
            plt.plot([landmarks_x[p1_idx], landmarks_x[p2_idx]], 
                     [landmarks_y[p1_idx], landmarks_y[p2_idx]], 
                     color=colors[idx], alpha=0.3)

    # Determine plot limits based on all plotted points
    all_x = [sample[f'landmark_{i}_x'] for _, sample in gesture_data.iterrows() for i in range(num_landmarks)]
    all_y = [sample[f'landmark_{i}_y'] for _, sample in gesture_data.iterrows() for i in range(num_landmarks)]
    if all_x and all_y:
        x_min, x_max = min(all_x), max(all_x)
        y_min, y_max = min(all_y), max(all_y)
        x_range = x_max - x_min
        y_range = y_max - y_min
        plt.xlim(x_min - 0.1 * x_range, x_max + 0.1 * x_range)
        plt.ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
    
    #plt.legend()
    plt.grid(True)
    plt.show()

# Plot landmarks for each gesture
for gesture in df_landmarks['gesture_label'].unique():
    plot_gesture_landmarks(df_landmarks, gesture, num_samples=10)

In [ ]:
# Visualize data using PCA
X = df_landmarks.drop('gesture_label', axis=1)
y = df_landmarks['gesture_label']

# Scale data before PCA
scaler_pca = StandardScaler()
X_scaled_pca = scaler_pca.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_pca)

pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['gesture_label'] = y.values # Use .values to avoid index mismatch if y's index was reset

plt.figure(figsize=(10, 8))
sns.scatterplot(x='PC1', y='PC2', hue='gesture_label', data=pca_df, alpha=0.7, s=50)
plt.title('PCA of Landmark Data (2 Components)')
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}% variance)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}% variance)')
plt.legend(title='Gesture')
plt.grid(True)
plt.show()

In [ ]:
print("Statistical Summary per Gesture:")
# Calculate mean position for a specific landmark (e.g., index fingertip) per gesture
index_tip_cols = ['landmark_8_x', 'landmark_8_y']
print(df_landmarks.groupby('gesture_label')[index_tip_cols].mean())

print("\nStandard Deviation per Gesture (Index Fingertip):")
print(df_landmarks.groupby('gesture_label')[index_tip_cols].std())

# Correlation matrix (might be too large to visualize well, show a subset)
subset_cols = ['landmark_4_x', 'landmark_4_y', 'landmark_8_x', 'landmark_8_y', 'landmark_0_x', 'landmark_0_y']
correlation_matrix = df_landmarks[subset_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Selected Landmark Coordinates')
plt.show()

In [ ]:
def calculate_distances(row):
    num_landmarks = 21
    landmarks = np.array([(row[f'landmark_{i}_x'], row[f'landmark_{i}_y']) for i in range(num_landmarks)])
    
    # Distances from fingertips to wrist (landmark 0)
    dist_thumb_tip_wrist = np.linalg.norm(landmarks[4] - landmarks[0])
    dist_index_tip_wrist = np.linalg.norm(landmarks[8] - landmarks[0])
    dist_middle_tip_wrist = np.linalg.norm(landmarks[12] - landmarks[0])
    dist_ring_tip_wrist = np.linalg.norm(landmarks[16] - landmarks[0])
    dist_pinky_tip_wrist = np.linalg.norm(landmarks[20] - landmarks[0])
    
    # Distances between adjacent fingertips
    dist_thumb_index = np.linalg.norm(landmarks[4] - landmarks[8])
    dist_index_middle = np.linalg.norm(landmarks[8] - landmarks[12])
    dist_middle_ring = np.linalg.norm(landmarks[12] - landmarks[16])
    dist_ring_pinky = np.linalg.norm(landmarks[16] - landmarks[20])
    
    return pd.Series([
        dist_thumb_tip_wrist, dist_index_tip_wrist, dist_middle_tip_wrist, dist_ring_tip_wrist, dist_pinky_tip_wrist,
        dist_thumb_index, dist_index_middle, dist_middle_ring, dist_ring_pinky
    ])

feature_names = [
    'dist_thumb_tip_wrist', 'dist_index_tip_wrist', 'dist_middle_tip_wrist', 'dist_ring_tip_wrist', 'dist_pinky_tip_wrist',
    'dist_thumb_index', 'dist_index_middle', 'dist_middle_ring', 'dist_ring_pinky'
]

# Apply the function row-wise
df_features = df_landmarks.apply(calculate_distances, axis=1)
df_features.columns = feature_names

# Combine engineered features with original data (optional, usually use features directly)
df_combined = pd.concat([df_landmarks['gesture_label'], df_features], axis=1)

print("Engineered Features Head:")
print(df_combined.head())

print("\nEngineered Features Description:")
print(df_combined.describe())

In [ ]:
# Explore the engineered features
plt.figure(figsize=(15, 12))
num_features = len(feature_names)
cols = 3
rows = math.ceil(num_features / cols)

for i, feature in enumerate(feature_names):
    plt.subplot(rows, cols, i + 1)
    sns.boxplot(data=df_combined, x='gesture_label', y=feature)
    plt.title(f'Box Plot of {feature}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Pairplot for a subset of features
sns.pairplot(df_combined, hue='gesture_label', vars=['dist_index_tip_wrist', 'dist_thumb_index', 'dist_ring_pinky'])
plt.suptitle('Pair Plot of Selected Engineered Features', y=1.02)
plt.show()

In [ ]:
# Prepare data for modeling using engineered features
X_feat = df_combined.drop('gesture_label', axis=1)
y_feat = df_combined['gesture_label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_feat, y_feat, test_size=0.3, random_state=42, stratify=y_feat)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set shape: {X_train_scaled.shape}")
print(f"Testing set shape: {X_test_scaled.shape}")

In [ ]:
# Initial Model Testing: K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Predictions
y_pred_knn = knn.predict(X_test_scaled)

# Evaluation
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Model Accuracy: {accuracy_knn:.4f}")

print("\nKNN Classification Report:")
print(classification_report(y_test, y_pred_knn))

print("\nKNN Confusion Matrix:")
cm_knn = confusion_matrix(y_test, y_pred_knn, labels=knn.classes_)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', xticklabels=knn.classes_, yticklabels=knn.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('KNN Confusion Matrix')
plt.show()

In [ ]:
# Initial Model Testing: Support Vector Machine (SVM)
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X_train_scaled, y_train)

# Predictions
y_pred_svm = svm.predict(X_test_scaled)

# Evaluation
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Model Accuracy: {accuracy_svm:.4f}")

print("\nSVM Classification Report:")
print(classification_report(y_test, y_pred_svm))

print("\nSVM Confusion Matrix:")
cm_svm = confusion_matrix(y_test, y_pred_svm, labels=svm.classes_)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens', xticklabels=svm.classes_, yticklabels=svm.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('SVM Confusion Matrix')
plt.show()